# Convert .Rdata files to .h5ad for viewing single cell data in cellxgene
code by Michelle Webb and Frank Grenn

[cellxgene](https://chanzuckerberg.github.io/cellxgene/)

our [FOUNDINPD github](https://github.com/FOUNDINPD/cellxgene-testing) with the cellxgene file conversion scripts

In [2]:
BIOWULF_DIR="/directory/with/data/files"
SCRIPT_DIR="/directory/for/scripts"
RDATA_FILENAME="FC_Set21_integrated_clustered"

## (1) Convert the .Rdata file to an .rds file

load the .Rdata file and list its object(s)



In [3]:
print("module load R")
print("R")
print("load('{}/{}.Rdata', verbose=TRUE)".format(BIOWULF_DIR,RDATA_FILENAME))
print("ls()")

module load R
R
load('/directory/with/data/files/FC_Set21_integrated_clustered.Rdata', verbose=TRUE)
ls()


run ```head()``` on object printed by ```ls()```  (```frontal.integrated``` in this case)

In [4]:
print("head(frontal.integrated)")
print("saveRDS(frontal.integrated, '{}/{}.rds')".format(BIOWULF_DIR,RDATA_FILENAME))

head(frontal.integrated)
saveRDS(frontal.integrated, '/directory/with/data/files/FC_Set21_integrated_clustered.rds')


## (2) Script to convert .rds to .loom

may need to install newer versions of ```Seurat``` and ```loomR``` libraries

### download updated R libraries on biowulf
https://hpc.nih.gov/apps/R.html

In [7]:
with open ("{}/seuratToLoom.R".format(SCRIPT_DIR), "w") as text_file:
    print("library(Seurat) \n\
library(loomR) \n\
integratedData <- readRDS('{}/{}.rds') \n\
DefaultAssay(integratedData) <- 'integrated' \n\
integratedData.loom <- as.loom(integratedData, filename='{}/{}.loom') \n\
integratedData.loom$close_all()".format(BIOWULF_DIR, RDATA_FILENAME, BIOWULF_DIR, RDATA_FILENAME), file = text_file)
    text_file.close()

## (3) Script to convert .loom to .h5ad
may need to install python loompy and scanpy packages

### (a) create custom python conda environment if you don't have loompy and scanpy setup

In [8]:
print("cd /data/$USER")
print("mkdir -p temp")
print("cd temp")

cd /data/$USER
mkdir -p temp
cd temp


In [7]:
print("wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh")

wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh


In [8]:
print("mkdir tmp")
print("TMPDIR=$PWD/tmp bash Miniconda3-latest-Linux-x86_64.sh -p /data/$USER/conda -b")

mkdir tmp
TMPDIR=$PWD/tmp bash Miniconda3-latest-Linux-x86_64.sh -p /data/$USER/conda -b


In [9]:
print("source /data/$USER/conda/etc/profile.d/conda.sh")
print("conda activate base")
print("which python")
print("conda update conda")
print("conda clean --all --yes")

source /data/$USER/conda/etc/profile.d/conda.sh
conda activate base
which python
conda update conda
conda clean --all --yes


In [10]:
print("conda create -n project1 python=3.7 numpy scipy")

conda create -n project1 python=3.7 numpy scipy


In [24]:
print("conda activate project1")
print("which pip")
print("pip install loompy")
print("pip install scanpy")
print("pip install cellxgene")
print("pip install cellxgene[prepare]")

conda activate project1
which pip
pip install loompy
pip install scanpy
pip install cellxgene
pip install cellxgene[prepare]


In [12]:
print("conda deactivate")

conda deactivate


### (b) write the script

need to rename fields in data.obsm to match cellxgene conventions:

like ```pca_cell_embeddings``` to ```X_pca``` for cellxgene

same for ```X_umap``` and ```X_tsne```

In [9]:
with open ("{}/create_h5ad.py".format(SCRIPT_DIR), "w") as text_file:
    print("import scanpy as sc \n\
import pandas as pd \n\
results_file = '{}/{}.h5ad' \n\
data = sc.read_loom('{}/{}.loom') \n\
# Rename seurat embeddings to match cellxgene convention \n\
data.obsm['X_pca'] = data.obsm.pop('pca_cell_embeddings') \n\
data.obsm['X_umap'] = data.obsm.pop('umap_cell_embeddings') \n\
data.write(results_file)".format(BIOWULF_DIR, RDATA_FILENAME, BIOWULF_DIR, RDATA_FILENAME), file = text_file)
    text_file.close()

## (4) Run scripts from (2) and (3) to convert the .rds to .h5ad

In [10]:
with open ("{}/rds_to_h5ad.sh".format(SCRIPT_DIR), "w") as text_file:
    print("#!/bin/bash \n\
module load R \n\
echo 'seurat to loom' \n\
Rscript {}/seuratToLoom.R \n\
source /data/grennfp/conda/etc/profile.d/conda.sh \n\
module load python/3.7 \n\
conda activate project1 \n\
echo 'loom to h5ad' \n\
python {}/create_h5ad.py \n\
echo 'done'".format(SCRIPT_DIR, SCRIPT_DIR), file = text_file)
    text_file.close()

run the script (may take a few hours)

In [5]:
print("sbatch --mem=100g --cpus-per-task=10 --mail-type=ALL --time=24:00:00 {}/rds_to_h5ad.sh".format(SCRIPT_DIR))

sbatch --mem=100g --cpus-per-task=10 --mail-type=ALL --time=24:00:00 /directory/for/scripts/rds_to_h5ad.sh


## (5) Load the .h5ad into cellxgene
run all of this locally

In [6]:
LOCAL_DIR="/local/directory/with/h5ad/files"

#### (a) pull the .h5ad down from biowulf

In [7]:
print("scp helix.nih.gov:{}/{}.h5ad {}".format(BIOWULF_DIR, RDATA_FILENAME, LOCAL_DIR))

scp helix.nih.gov:/directory/with/data/files/FC_Set21_integrated_clustered.h5ad /local/directory/with/h5ad/files


#### (b) install cellxgene locally if not done already

In [24]:
print("pip install cellxgene")

pip install cellxgene


#### (c) run cellxgene

In [8]:
print("cellxgene launch {}/{}.h5ad --open".format(LOCAL_DIR, RDATA_FILENAME))

cellxgene launch /local/directory/with/h5ad/files/FC_Set21_integrated_clustered.h5ad --open


## (6) Other

### to load and view the .h5ad file in python

In [2]:
print("import anndata")
print("data=anndata.read_h5ad('/path/to/file.h5ad')")
print("data")

import anndata
data=anndata.read_h5ad('/path/to/file.h5ad')
data


### write the .h5ad file

In [3]:
print("data.write('/path/to/newfile.h5ad')")

data.write('/path/to/newfile.h5ad')
